In [100]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(1, 51)
        self.lstm2 = nn.LSTMCell(51, 51)
        self.linear = nn.Linear(51, 1)

    def forward(self, input, future = 0, seq_len = 32):
        outputs = []
        h_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 51, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 51, dtype=torch.double)

        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            output = self.linear(h_t)
            outputs += [output]
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            output = self.linear(h_t)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs

In [101]:
def make_data(directory):
    import os
    directory = "./train_data"
    xTr = []
    yTr = []
    for filename in os.listdir(directory):
        beats = np.load(directory + os.sep + filename)
        x, y = split_data(beats)
        xTr.append(x)
        yTr.append(y)
    xTr = np.concatenate(np.array(xTr))
    yTr = np.concatenate(np.array(yTr))
    
    
def split_data(beats, data_len=32):
    end = beats.shape[0]- (beats.shape[0]%(2*data_len))
    beats = beats[:end].reshape(-1, 2*data_len)
    x, y = beats[:,:32], beats[:,32:]
    return x, y

In [102]:
print(xTr.shape, yTr.shape)

(9861, 32) (9861, 32)


In [ ]:
n_epochs = 100
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
nesterov = True

# set random seed to 0
np.random.seed(0)
torch.manual_seed(0)

# load data and make training set
input = torch.from_numpy(xTr).double()
target = torch.from_numpy(yTr).double()
test_input = input[:100]
test_target = target[:100]

# build the model
seq = Sequence()
seq.double()
criterion = nn.MSELoss()

optimizer = optim.SGD(seq.parameters(), lr=lr, momentum=momentum, nesterov=nesterov, weight_decay=weight_decay)

#begin to train
for i in range(n_epochs):
    print('STEP: ', i)
    
    def closure():
        optimizer.zero_grad()
        out = seq(input)
        loss = criterion(out, target)
        print('loss:', loss.item())
        loss.backward()
        return loss
    
    optimizer.step(closure)
    
    # begin to predict, no need to track gradient here
    with torch.no_grad():
        future = 32
        pred = seq(test_input, future=future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.item())
        y = pred.detach().numpy()
        
    # draw the result
    if i % 10 == 0:
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        plt.savefig('predict%d.png'%i, bbox_inches='tight')
        plt.close()


STEP:  0
